## Implement context-sensitive spelling correction

In [1]:
from collections import Counter
from tqdm import tqdm
import pandas as pd


class Bigrams:
    def __init__(self, main_path, additional_path):
        # read files
        self.gram_main = list(pd.read_csv(main_path).text)
        self.gram_add = pd.read_csv(additional_path, header=None, sep=";")

        # create hash tables of words and bigrams
        self.word = {}
        self.bigram = {}

        # call function to generate hash tables
        self.__transform_main()
        self.__generate_dict()

        # clear memory
        self.gram_main = None
        self.gram_add = None

    def __transform_main(self) -> None:
        """This function count bigrams in main_file"""
        self.gram_main = [b for l in self.gram_main for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
        self.gram_main = Counter(self.gram_main)

    def __generate_dict(self) -> None:
        """Transform DFs to dictionary of words and bigrams"""
        for i in tqdm(range(self.gram_add.shape[0])):
            count = self.gram_add.iloc[i][0]
            first = self.gram_add.iloc[i][1]
            second = self.gram_add.iloc[i][2]

            self.word[first] = self.word[first] + count if first in self.word else count
            self.word[second] = self.word[second] + count if second in self.word else count
            self.bigram[f"{first} {second}"] = count

        for ws in tqdm(self.gram_main.keys()):
            first, second = list(ws)
            count = self.gram_main[ws]

            self.word[first] = self.word[first] + count if first in self.word else count
            self.word[second] = self.word[second] + count if second in self.word else count
            self.bigram[f"{first} {second}"] = self.bigram[
                                                   f"{first} {second}"] + count if f"{first} {second}" in self.bigram else count

    def is_word_exist(self, word: str) -> bool:
        return word in self.word

    def count_bigram(self, word1: str, word2: str) -> int:
        bi = word1 + " " + word2
        return self.bigram[bi] if bi in self.bigram else 0

    def count_word(self, w: str) -> int:
        return self.word[w]



In [2]:
import itertools


class ContextSpellChecker:
    def __init__(self, bigram: Bigrams, max_depth=2, lower_fr=100, trade_of = 100):
        self.bgr = bigram
        self.max_depth = max_depth
        self.lower_fr = lower_fr
        self.trade_of = trade_of

    def edit(self, word):
        """Returns all variants of candidate words with a distance equal to 1, using Norvig part of code"""
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return list(set(deletes + transposes + replaces + inserts))

    def merge_candidates(self, c1: list, c2: list):
        """Create all pairs of elements betwean two arrays"""
        return [list(pair) for pair in list(itertools.product(c1, c2))]

    def get_candidates(self, w: str, depth: int):
        """Create all candidates of givven word with distance n"""
        can = [w]
        for i in range(depth):
            t1 = [self.edit(i) for i in can]
            for c in t1: can += c
            can = list(set(can))
        return [w for w in list(set(can)) if self.bgr.is_word_exist(w)]

    def get_info(self, s: str) -> tuple:
        """Split original string by spaces and create arrays of position correct and incorect words"""
        splits = s.split()

        # ad "_" as end/start of string
        signature = ["_"]
        for w in splits:
            # create array like ["-", "v", "x", "_"] where v - correcct, x - with mistake
            if self.bgr.is_word_exist(w) and self.bgr.count_word(w) > self.lower_fr:
                signature.append("v")
            else:
                signature.append("x")
        signature.append("_")

        splits = ["_"] + splits + ["_"]

        return splits, signature

    def best_count(self, w: str) -> str:
        """Return best condidate by freqyency of using"""
        for d in range(1, self.max_depth + 1):

            error = self.get_candidates(w, depth=d)
            fin = [[i, self.bgr.count_word(i)] for i in error]

            if fin != []:
                break

        return  [[max(fin, key=lambda x: x[1])[0]],
                max(fin, key=lambda x: x[1])[1]/self.trade_of]\
            if fin != []  and max(fin, key=lambda x: x[1])[1] != 0 else\
            [[w], 1]

    def best_previous(self, w: str, c: str) -> list:
        "return best word using correct next word or by frequency if bigram doesn't exist"
        for d in range(1, self.max_depth + 1):

            error = self.get_candidates(w, depth=d)
            can = self.merge_candidates(error, [c])
            fin = [[i, self.bgr.count_bigram(*i)] for i in can]

            if fin != [] and max(fin, key=lambda x: x[1])[1] != 0:
                break
        return max(fin, key=lambda x: x[1]) \
            if fin != [] and max(fin, key=lambda x: x[1])[1] != 0 else\
            self.best_count(w)

    def best_next(self, c: str, w: str) -> list:
        "return best word using correct previous word or by frequency if bigram doesn't exist"
        for d in range(1, self.max_depth + 1):

            error = self.get_candidates(w, depth=d)
            can = self.merge_candidates([c], error)
            fin = [[i, self.bgr.count_bigram(*i)] for i in can]

            if fin != [] and max(fin, key=lambda x: x[1])[1] != 0:
                break

        return max(fin, key=lambda x: x[1]) \
            if fin != [] and max(fin, key=lambda x: x[1])[1] != 0 else \
            self.best_count(w)

    def best_gram(self, w1: str, w2: str) -> list:
        """Search best fit ngramor or return best by frequency if bigram doesn't exist"""
        for d in range(1, self.max_depth + 1):
            error1 = self.get_candidates(w1, depth=d)
            error2 = self.get_candidates(w2, depth=d)
            can = self.merge_candidates(error1, error2)
            fin = [[i, self.bgr.count_bigram(*i)] for i in can]

            if fin != [] and max(fin, key=lambda x: x[1])[1] != 0:
                break

        return max(fin, key=lambda x: x[1]) \
            if fin != [] and max(fin, key=lambda x: x[1])[1] != 0 else \
            [[self.best_count(w1)[0][0], self.best_count(w2)[0][0]], 1]

    def get_string(self, st: str) -> str:
        s, sig = self.get_info(st)

        # if all words in string are correct
        if sig.count("x") == 0:
            return st

        #looking all mistakes in string
        while "x" in sig:
            i = sig.index("x")

            #process different patern of the relative positions of words
            match sig[i - 1] + sig[i] + sig[i + 1]:
                case "_x_":
                    s[i] = self.best_count(s[i])[0][0]
                    sig[i] = "v"
                case "_xv":
                    s[i] = self.best_previous(s[i], s[i+1])[0][0]
                    sig[i] = "v"
                case "vx_":
                    var = self.best_next(s[i-1], s[i])
                    s[i] = self.best_next(s[i-1], s[i])[0][1] if len(var[0]) == 2 else self.best_next(s[i-1], s[i])[0][0]
                    sig[i] = "v"
                case "vxv":
                    choose = [self.best_next(s[i-1], s[i]), self.best_previous(s[i], s[i+1])]
                    best = max(choose, key=lambda x: x[1])
                    s[i] = best[0][0] \
                        if len(best[0]) == 1 else \
                        best[0][choose.index(best) == 0]
                    sig[i] = "v"
                case "_xx":
                    s[i], s[i+1] = self.best_gram(s[i], s[i+1])[0]
                    sig[i], sig[i+1] = "v", "v"
                case "vxx":
                    choose = [self.best_next(s[i-1], s[i]), self.best_gram(s[i], s[i+1])]
                    if choose[0][1] <= choose[1][1]:
                        s[i], s[i+1] = choose[1][0]
                        sig[i], sig[i+1] = "v", "v"
                    else:
                        s[i] = choose[0][0][0] if len(choose[0][0]) == 1 else choose[0][0][1]
                        sig[i] = "v"
        return " ".join(s[1:-1:])


## Justify your decisions

This solution is an improvement on Norvig's solution. This solution uses:
 - Dynamic depth of candidate search
  - If a word is used more often than a certain amount, it is considered to exist and is not investigated for coreection.
- To find a word in a suitable context, the Markov assumption is used and only the neighbors to the right and left of the word are considered, bigrams were used for this(Compiled from training data and files attached to the task).
- Words and diagrams data structure is hash tables(dictionaries) since the search in them is carried out in O(1)
- Work with all string, not only one word

[Link for dataset](https://www.kaggle.com/datasets/samarthagarwal23/spelling-mistake-data-1mn/data?select=train.csv)

Parameters:
- max_depth - what are the max distance betwean original word and candidates
- lower-fr - The boundary of how a part of a word is used in the data to consider it as existing.
- trade_of - Coefficient which spelling or context to trust more (0; +inf)

## Evaluate on a test set

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
bigram = Bigrams("train.csv", "bigrams.csv")

100%|██████████| 1748336/1748336 [00:03<00:00, 513461.49it/s]


In [5]:
csch = ContextSpellChecker(bigram, max_depth=2, lower_fr=100, trade_of=100)

In [6]:
total_test_size = 300
correct = 0
test = pd.read_csv("test.csv")[0:total_test_size]
test

,text,augmented_text
0,project looks to mulesing genetic alternative,project looks to muelsnig ngeetic alternative
1,chemical agents used during protest at port au...,chemical agents used during LrotWst at port ah...
2,business chamber seeks budget infrastructure b...,business hcmaber seeks budget infrastrcutuer b...
3,3600 trips made to darwin tip after cyclone,3600 trips made to adrwni tip after cyconle
4,go between bridge to open july 5,go net3een brisye to lprn july 5
...,...,...
295,council seeks job support for walkers workers,council seeks job spupotr for walkrse wroekrs
296,quarantine officers find bizarre items,quarQBtine obflcers find bizarre items
297,support aired for indigenous language funds,suppotr aired for indigenous language ufnsd
298,deep fryer thought to have sparked manildra hotel,deep f3yeG thlughR to have sparked manildra hpteP


In [7]:
for row in tqdm(range(total_test_size)):
    right = test.text[row]
    original = test.augmented_text[row]
    correct += csch.get_string(original) == right

100%|██████████| 300/300 [02:32<00:00,  1.96it/s]


In [8]:
print(f"Accuracy = {round((correct/total_test_size) * 100, 2)}%")

Accuracy = 46.67%


## Norvig solution

In [22]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [23]:
correct_n = 0
for row in tqdm(range(total_test_size)):
    right = test.text[row].split()
    original = test.augmented_text[row].split()
    t = []
    for i in original:
        t.append(correction(i))
    correct_n += t==right

100%|██████████| 300/300 [00:42<00:00,  7.12it/s]


In [24]:
print(f"Accuracy = {round((correct_n/total_test_size) * 100, 2)}%")

Accuracy = 9.33%


# Conclution

These tests check whether the algorithms correct the entire string correctly. Tests have shown that a context-sensitive solution copes with the task much better. Norvig solution is faster in 3.85 times, but worse in 5 times.